# Preprocess

#### Data Cleaning Steps
* remove number with a sign : , ( ) - + 
* may deal with wrong spelling words
* remove stopwords

# Process
#### Steps
* Clustering 
* Topic modeling
* Subcluster

In [16]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import pythainlp
from pythainlp import word_tokenize, Tokenizer
from pythainlp.corpus.common import thai_stopwords
from pythainlp.ulmfit import document_vector
import re

In [2]:
data = pd.read_excel('Abstract.xlsx')
print(data.columns)

Index(['ชื่อเรื่องไทย', 'ชื่อเรื่องอังกฤษ', 'คำสำคัญไทย', 'คำสำคัญอังกฤษ',
       'บทคัดย่อ', 'Abstract'],
      dtype='object')


In [3]:
# abstract = ''
# for i in range(0, 10):
#   abstract += data.at[i,'บทคัดย่อ']
# print(abstract)

abstract = []
for i in range(0,9):
    sub = [data.at[i,'บทคัดย่อ']]
    abstract.append(sub)

In [4]:
# print(abstract)

In [53]:
def remove(text):
    text = text.replace("-",'')
    return re.sub(r"[\n|\t|\d|%|(|)|,|.|/]{1,}", "", text)

rm_abstract = []
for i in range(0,len(abstract)):
    sub = [remove(abstract[i][0])]
    rm_abstract.append(sub)

In [54]:
# print(rm_abstract)

In [55]:
token_abstract = []
for i in range(0,len(abstract)):
    sub = word_tokenize(rm_abstract[i][0], engine="deepcut", keep_whitespace=False)
    token_abstract.append(sub)

In [56]:
# print(token_abstract)

In [57]:
sw_abstract = []
sub = []
for i in range(0,len(abstract)):
    for j in range(0,len(token_abstract[i])):
        word = token_abstract[i][j]
        if word not in thai_stopwords():
            sub.append(word)
    sw_abstract.append(sub)
    sub = []

In [58]:
print(sw_abstract)

[['เพาะ', 'เลี้ยง', 'อับละออง', 'เรณู', 'สร้าง', 'ประชากร', 'แผนที่', 'พันธุกรรม', 'สร้าง', 'สาย', 'พันธุ์'], ['ศัตรู', 'ข้าว', 'ผสมผสาน', 'แก้', 'ปัญหา', 'ระบาด', 'ศัตรู', 'ข้าว', 'ต่อเนื่อง', 'นาชลประทาน', 'ดำเนินการ', 'แปลง', 'นา', 'เกษตรกร', 'เขต', 'พื้นที่', 'จังหวัด', 'จังหวัดพัทลุง', 'สงขลา', 'นครศรีธรรมราช', 'เวลา', 'ปี', 'ฤดู', 'ฤดู', 'นาปรัง', 'พศ', 'ฤดู', 'นาปี', 'พศ', 'ข้อมูล', 'จำนวน', 'ประชากร', 'แมลง', 'ศัตรู', 'ข้าว', 'ศัตรู', 'ธรรมชาติ', 'เปอร์เซ็นต์', 'ทำลาย', 'โรค', 'ข้าว', 'สัตว์', 'ศัตรู', 'ข้าว', 'ชนิด', 'น้ำหนัก', 'สด', 'วัชพืช', 'ผลิต', 'ไร่', 'ต้นทุน', 'ผลิต', 'ราคา', 'ผลิต', 'ตอบแทน', 'สุทธิ', 'เปรียบเทียบ', 'ศัตรู', 'ข้าว', 'เกษตรกร', 'ทดลอง', 'แปลง', 'ศัตรู', 'ข้าว', 'ผสมผสาน', 'จำนวน', 'ศัตรู', 'ธรรมชาติ', 'แปลง', 'เกษตรกร', 'ระบาด', 'ศัตรู', 'ข้าว', 'เพลี้ยไฟ', 'เพลี้ยจักจั่น', 'สี', 'เขียว', 'หนอนห่อ', 'ใบ', 'ข้าว', 'เพลี้ยจักจั่น', 'ปี', 'กลาย', 'หยัก', 'เพลี้ย', 'ก', 'ระโดด', 'ขาว', 'แมลงสิง', 'หนู', 'หอยเชอรี่', 'โรค', 'ไหม้', 'โรค', 'ใบ', 'จุด', 'สี',

## Numbericalization

use Thai2fit cr. https://www.thainlp.org/pythainlp/tutorials/notebooks/word2vec_examples.html
https://github.com/cstorm125/thai2fit/

In [59]:
from pythainlp.word_vector import sentence_vectorizer
w = sentence_vectorizer(sw_abstract[0][0], use_mean=True)
print(w[0])

[ 0.074935 -0.06831   0.09751  -0.024656 ... -0.44411  -0.084777 -0.015924 -0.042845]


In [61]:
num_abstract = []
sub = []
subsub = {}
for i in range(0,len(abstract)):
    for j in range(0,len(token_abstract[i])):
        word = sw_abstract[i][j]
        num = sentence_vectorizer(sw_abstract[i][j], use_mean=True)
        subsub = {word : num}
        sub.append(subsub)
    num_abstract.append(sub)
    sub = []

IndexError: list index out of range

In [ ]:
#stolen from https://blog.manash.me/how-to-use-pre-trained-word-vectors-from-facebooks-fasttext-a71e6d55f27
def plot_with_labels(low_dim_embs, labels, filename=None, figsize=(10,10),
                    axis_lims = None):
    assert low_dim_embs.shape[0] >= len(labels), "More labels than embeddings"
    plt.figure(figsize=figsize)  # in inches
    for i, label in enumerate(labels):
        x, y = low_dim_embs[i, :]
        plt.scatter(x, y)
        prop = fm.FontProperties(fname=f'THSarabunNew.ttf',size=20)
        plt.annotate(label,
                 fontproperties=prop,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')
    if axis_lims is not None: plt.axis(axis_lims)
    if filename: plt.savefig(filename)

plot_with_labels(thai2plot[200:500],labels[200:500],axis_lims = [0,30,0,30])